<a href="https://colab.research.google.com/github/JavZon/Machine_Learning/blob/main/Random_Forest_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn

Datasetni chaqiramiz

In [2]:
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)

Datasetni 2 qismga bo'lamiz

In [3]:
from sklearn.model_selection import train_test_split
train_set, test_set=train_test_split(df,test_size=0.2, random_state=42)

Qismlarga ajratamiz

In [4]:
X_train = train_set.drop(columns='median_house_value')

y = train_set["median_house_value"].copy() # faqat labellarini saqlab oldik

X_num = X_train.drop(columns='ocean_proximity')

Create a Transformer

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
      pass
    def fit(self, X, y=None):
      return self
    def transform(self, X):
      rooms_per_household=X[:, rooms_ix] / X[:, households_ix]
      population_per_household=X[:, population_ix] / X[:, households_ix]
      return np.c_[X, rooms_per_household, population_per_household]

Pipeline for numerical columns

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

numeric_pipeline=Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('custom_obj_adder', CustomTransformer()),
    ('std_scaler', StandardScaler())
])

prepared_num=numeric_pipeline.fit_transform(X_num)
prepared_num[0,:]

array([ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
        0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609])

Final pipeline ( + category pipeline)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_cols=list(X_num.columns)
string_cols=['ocean_proximity']

full_pipeline=ColumnTransformer([
    ('numeric', numeric_pipeline, num_cols),
    ('string_cat',OneHotEncoder(handle_unknown='ignore'), string_cols)
])

Our final array ready to use (X_prepared)

In [8]:
X_prepared=full_pipeline.fit_transform(X_train)

X_prepared[:3,:]

array([[ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
         0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813,  0.34029326,  0.59309419,
        -0.09890135,  0.6720272 , -0.03584338, -0.40283542, -0.11736222,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, -0.34259695, -0.49522582,
        -0.44981806, -0.43046109,  0.14470145,  0.08821601, -0.03227969,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ]])

#**Random Forest** Algorithm

In [10]:
from sklearn.ensemble import RandomForestRegressor

Creating our own model called RF_model

In [11]:
RF_model=RandomForestRegressor()

We will give our X_prepared to our model and fit it

In [12]:
RF_model.fit(X_prepared, y)

RandomForestRegressor()

In [16]:
X_sample=X_train.sample(7)
Y_sample_label=y.loc[X_sample.index]

In [17]:
X_sample_prepared=full_pipeline.transform(X_sample)
X_sample_prepared

array([[-1.00604147,  1.94554414, -1.47659983, -0.41387505, -0.55250582,
        -0.62043921, -0.49346042, -0.7494103 ,  0.06203584, -0.06789057,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.5845107 , -0.78308938,  0.66589722, -0.46583909, -0.62887916,
        -0.66705221, -0.61420914,  1.18291296,  0.288537  , -0.05058413,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.22821659, -0.43206448, -1.2385446 ,  0.59781414,  0.11098752,
         0.10162249,  0.03940892,  1.43083387,  0.92955704, -0.00887744,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.69919001, -0.75968772,  0.34849025, -0.8323465 , -0.74582583,
        -0.38649474, -0.73495785, -0.41147954, -0.69258788,  0.11999959,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-1.12570684,  0.60696921,  0.03108328,  1.04801584,  1.0226942 ,
         0.7849866 ,  0.84790033,  0.56240172, 

In [18]:
X_sample_predicted=RF_model.predict(X_sample_prepared)
pd.DataFrame({'Predicted':X_sample_predicted, 'Real values':Y_sample_label})

,Predicted,Real values
1106,113171.00,117900.0
5526,388299.03,387200.0
17324,440941.63,500001.0
7206,133668.00,125000.0
18678,293406.00,307900.0
6112,146549.00,136100.0
11371,225934.00,229600.0


Assigning **test_set**

In [20]:
X_test_set=test_set.drop('median_house_value',axis=1) # test setning ham labelini olib tashladik
Y_test_set_label=test_set['median_house_value'].copy()

Pipelinedan o'tkazamiz

In [21]:
X_test_set_prepared=full_pipeline.transform(X_test_set)

**Predict** X_test_set_prepared

In [25]:
X_test_predicted=RF_model.predict(X_test_set_prepared)

In [26]:
X_test_predicted

array([ 47252.  ,  68911.  , 470242.46, ..., 500001.  ,  70887.  ,
       176494.  ])

We will calculate **RMSE** and **MAE**

In [27]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

mae=mean_absolute_error(Y_test_set_label,X_test_predicted)  # calculate mae
print("MAE -",mae)

mse=mean_squared_error(Y_test_set_label,X_test_predicted)
print("RMSE -",np.sqrt(mse))

MAE - 31816.461751453488
RMSE - 49574.47292327645
